In [1]:
import os
import pandas as pd
import xlrd
from pathlib import Path
import openpyxl
from openpyxl import load_workbook
import numpy as np
import re
import ast
from xls2xlsx import XLS2XLSX

In [2]:

    ##Getting the 1045 data
xlsx_file = Path('SimData', '/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster/Alvarado Hospital/106370652_CDM_All_2020.xlsx')
wb_obj = openpyxl.load_workbook(xlsx_file) 

In [3]:
#Function to extract 1045 sheet name from xlsx file only

def find1045_xlsx(filepath):
    #read file
    xlsx_file = Path(filepath)
    wb_obj = openpyxl.load_workbook(xlsx_file)

    #get sheet names
    sheetnames=wb_obj.sheetnames

    #Create a keywords list to identify ABS 1045 forms
    keywords= ["common", "procedures", "1045", "form"]

    #Search for the sheet by keyword
    for name in sheetnames:
        wholename=name.lower()
        wordsInName=wholename.split()
        match =  any(item in wordsInName for item in keywords)
        if match==True:
            sheetname=(wb_obj[name])
            return(sheetname)
            break


In [4]:
#Function to extract all dataframes from a single xlsx file
def getdata_xlsx(file):
    dflist=[]
    coordlist=[]
    sheet=find1045_xlsx(file)
    alphabet=["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
    found=False
    keywords1 = ["evaluation", "management", "services"]
    keywords2 = ["laboratory", "pathology", "services"]
    keywords3= ["radiology", "services"]
    keywords4 = ["medicine", "services"]
    keywords5 = ["surgery", "services"]
    keywords6 = ["other", "common", "outpatient", "procedures"]
    keywords7 = ["count", "of", "reported", "procedures",]
    keywordslists=[keywords1,keywords2,keywords3,keywords4,keywords5,keywords6,keywords7]
    for wordlist in keywordslists:
        for row in sheet.iter_rows():
                for cell in row:
                    heading = str(cell.value).lower()
                    wordsInHeading=heading.split()
                    match =  all(item in wordsInHeading for item in wordlist)
                    if match==True:
                        coord1=cell.coordinate
                        coordlist.append(coord1)
                        break
    for coordinate in coordlist[0:-1]:
        splitcoord=re.split('(\d+)', coordinate)
        nextbreak=coordlist[int(coordlist.index(coordinate)+1)]
        splitbreak=re.split('(\d+)', nextbreak)
        lastcell=f"{int(splitbreak[1])-1}"
        coord2=f"C{lastcell}"

        cellgroup=[coordinate,coord2]
        df=pd.DataFrame(sheet[f"{cellgroup[0]}:{cellgroup[1]}"])



        dflist.append(df)

    return(dflist)

  
    

In [5]:
#Function to convert cell object values to plaintext values
def convertvalue(x):
    return x.value

In [6]:
#Convert all dataframes from a single sheet into plaintext
sheet=find1045_xlsx(xlsx_file)
masterlist=[]
rawlist=getdata_xlsx(xlsx_file)
for i in range(len(rawlist)):
    df=rawlist[i].applymap(convertvalue)
            #add the hospital name as a variable
    masterlist.append(df)

In [7]:
#Get the column names from the dataframe
for df in masterlist:
    if len(df)>0:
        df.columns = df.iloc[0]
        df["Service Type"]=df.iloc[0,0]
        df.rename(columns={ df.columns[0]: "Service" }, inplace = True)
        df=df.drop(df.index[0], inplace=True)




In [8]:
#add the hospital name as a variable in every df in the converted list
for i in range(len(masterlist)):
        raw_name=sheet["A1"].value
        hos_name=raw_name.replace('Hospital Name: ', '')
        masterlist[i]["Hospital"]=hos_name


In [9]:
masterlist[0]

,Service,2019 CPT Code,Average Charge,Service Type,Hospital
1,"Emergency Room Visit, Level 2 (low to moderate...",99282,746.55,Evaluation & Management Services (CPT Codes 99...,Alvarado Hospital
2,"Emergency Room Visit, Level 3 (moderate severity)",99283,1051.05,Evaluation & Management Services (CPT Codes 99...,Alvarado Hospital
3,"Emergency Room Visit, Level 4 (high severity)",99284,1581.3,Evaluation & Management Services (CPT Codes 99...,Alvarado Hospital
4,"Outpatient Visit, established patient, 15 minutes",99213,None,Evaluation & Management Services (CPT Codes 99...,Alvarado Hospital


In [10]:
#get the names of all folders in the data directory
os.chdir("..")
os.chdir("data")
os.chdir("2020_CDM_chargemaster")



In [11]:
#Function to extract 1045 sheet name

def find1045(filepath):
    #read file
    if filepath[-5:] == ".xlsx":
        excel_file = Path(filepath)
        wb_obj = openpyxl.load_workbook(excel_file)

    #get sheet names
    sheetnames=wb_obj.sheetnames

    #Create a keywords list to identify ABS 1045 forms
    keywords= ["common", "procedures", "1045", "form"]

    #Search for the sheet by keyword
    for name in sheetnames:
        wholename=name.lower()
        wordsInName=wholename.split()
        match =  any(item in wordsInName for item in keywords)
        if match==True:
            sheetname=(wb_obj[name])
            return(sheetname)
            break


In [12]:
#get the names of all files from each directory, then test if each file has a 1045 in it.
dirlist = list(os. listdir())
for i in range(len(dirlist)):
    os.chdir(dirlist[i])
    for root, dirs, files in os.walk("."):
        for filename in files:
            print(filename)
            print (find1045_xlsx(filename))
    os.chdir("..")

106390923_CDM_All_2020.xlsx
<Worksheet "Common OP Procedures">
106104023_CDM_All_2020.xls


InvalidFileException: openpyxl does not support the old .xls file format, please use xlrd to read this file, or convert it to the more recent .xlsx file format.

In [ ]:
filename="test.xlsx"

In [ ]:
filename[-5:]

In [15]:
os.chdir("data")

FileNotFoundError: [Errno 2] No such file or directory: 'data'

In [16]:
os.getcwd()

'/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster/San Joaquin Valley Rehab Center'

In [17]:
os.chdir("..")

In [18]:
os.chdir("2020_CDM_chargemaster")

FileNotFoundError: [Errno 2] No such file or directory: '2020_CDM_chargemaster'

In [2]:

    ##Getting the 1045 data
xls_file = Path("/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster/Emanate Health Foothill Presbyterian Hospital/106190298_Common25_2020.xls")

In [20]:
book = xlrd.open_workbook(xls_file)

In [21]:
book = xlrd.open_workbook(xls_file)

print("The number of worksheets is {0}".format(book.nsheets))
print("Worksheet name(s): {0}".format(book.sheet_names()))
sh = book.sheet_by_index(0)
print("{0} {1} {2}".format(sh.name, sh.nrows, sh.ncols))
print("Cell D30 is {0}".format(sh.cell_value(rowx=29, colx=3)))
for rx in range(sh.nrows):
    print(sh.row(rx))

The number of worksheets is 2
Worksheet name(s): ['Top 50 List', 'AB 1045 Form']
Top 50 List 51 3


IndexError: list index out of range

In [27]:
#Function to extract 1045 sheet from xls file (not name, whole sheet)

def find1045_xls(filepath):
    #read file
    xls_file = Path(filepath)
    book = xlrd.open_workbook(xls_file)

    #get sheet names
    sheetnames=("{0}".format(book.sheet_names()))
    sheetnames= ast.literal_eval(sheetnames)


#Create a keywords list to identify ABS 1045 forms
    keywords= ["common", "procedures", "1045", "form"]

#Search for the sheet by keyword
    for name in sheetnames:
        wholename=name.lower()
        wordsInName=wholename.split()
        match =  any(item in wordsInName for item in keywords)
        if match==True:
            cur_sheet = book.sheet_by_name(name)
            return(cur_sheet)
            break


In [35]:
x2x = XLS2XLSX(xls_file)


AttributeError: 'PosixPath' object has no attribute 'read'

In [36]:
from xls2xlsx import XLS2XLSX
x2x = XLS2XLSX(xls_file)
wb = x2x.to_xlsx()

AttributeError: 'PosixPath' object has no attribute 'read'

In [ ]:
884

>>> import ast
>>> x = '[ "A","B","C" , " D"]'
>>> x = ast.literal_eval(x)
>>> x
['A', 'B', 'C', ' D']
>>> x = [n.strip() for n in x]
>>> x
['A', 'B', 'C', 'D']

In [ ]:

import xlrd
from openpyxl.workbook import Workbook as openpyxlWorkbook

# content is a string containing the file. For example the result of an http.request(url).
# You can also use a filepath by calling "xlrd.open_workbook(filepath)".

xlsBook = xlrd.open_workbook(file_contents=content)
workbook = openpyxlWorkbook()

for i in xrange(0, xlsBook.nsheets):
    xlsSheet = xlsBook.sheet_by_index(i)
    sheet = workbook.active if i == 0 else workbook.create_sheet()
    sheet.title = xlsSheet.name

    for row in xrange(0, xlsSheet.nrows):
        for col in xrange(0, xlsSheet.ncols):
            sheet.cell(row=row + 1, column=col + 1).value = xlsSheet.cell_value(row, col)

# The new xlsx file is in "workbook", without iterators (iter_rows).
# For iteration, use "for row in worksheet.rows:".
# For range iteration, use "for row in worksheet.range("{}:{}".format(startCell, endCell)):".

In [98]:
#Iterate through files and convert all xls files to xlsx, removing the original file
def convert(filename):
    p.save_book_as(file_name=f"{filename}",
                dest_file_name=f"{filename[:-4]}.xlsx")

In [4]:
from pyexcel_xls import get_data
data = get_data(xls_file)


Exception: file_name should be a string

In [54]:
cwd=os.getcwd()

In [56]:
os.getcwd()

'/home/ygael/Projects/healthcare/data/2019_CDM_chargemaster'

In [52]:
os.chdir("..")

In [53]:
cwd

'/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster'

In [15]:
os.chdir("data")

In [17]:
os.chdir("2020_CDM_chargemaster")

In [99]:
#A function to convert all files in subfolders starting from the absolute filepath of the year's chargemaster folder

def convertfrom(directory):
    #Use absolute path to move to the directory containing all hospital folders
    os.chdir(directory)
    #save current working directory to a variable
    cwd=os.getcwd()
    #Convert all files to xlsx format
    dirlist = list(os. listdir())
    for i in range(len(dirlist)):
        os.chdir(dirlist[i])
        for root, dirs, files in os.walk("."):
            for filename in files:
                if filename[-3:]=="xls":
                    convert(filename)
                    os.remove(filename)
                else:
                    print(filename)
                    print("no")
        os.chdir("..")
    os.chdir(cwd)

In [100]:
convertfrom("/home/ygael/Projects/healthcare/data/2019_CDM_chargemaster")

106331194_Common25_2019.xlsx
no
106331194_CDM_2019.xlsx
no
Thumbs.db
no
106331194_PCT_CHG_2019.xlsx
no
106301258_CDM_2019.xlsx
no
106301258_PCT_CHG_2019.xlsx
no
106301258_Common25_2019.xlsx
no
106190382_CDM_2019.xlsx
no
106190382_Common25_2019.xlsx
no
Thumbs.db
no
106190382_PCT_CHG_2019.xlsx
no
106390923_CDM_All_2019.xlsx
no
106104023_CDM_All_2019.xls.xlsx
no
106370730_PCT_CHG_2019.xlsx
no
106374465_Common25_2019.xlsx
no
106370730_Common25_2019.xlsx
no
Thumbs.db
no
106370730_CDM_2019.xlsx
no
106361339_CDM_All_2019.xlsx
no
Thumbs.db
no
106314024_Common25_2019.xlsx
no
106314024_PCT_CHG_2019.xls.xlsx
no
Thumbs.db
no
2019 NCAL CDM Final.xlsx
no
106560473_CDM_All_2019.xlsx
no
106190197_Common25_2019.xlsx
no
106190197_Comments_2019.docx
no
106190197_CDM_2019.xlsx
no
106331293_CDM_All_2019.xlsx
no
106015000_Common25_2019.xlsx
no
106015000_CDM_2019.xlsx
no
Thumbs.db
no
106015000_PCT_CHG_2019.xls.xlsx
no
106391010_CDM_2019.xlsx
no
106391010_Comments_2019.doc
no
106391010_Common25_2019.xlsx
no
1

IllegalCharacterError: 

In [70]:
from import_export import resources
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE

class CleanModelResource(resources.ModelResource):
  def export_field(self, field, obj):
    v = super(CleanModelResource, self).export_field(field, obj)
    if type(v) == str:
      v = ILLEGAL_CHARACTERS_RE.sub('', v)
    return v